In [1]:
# Duplicate this template, write your scryfall query here and then run the notebook
QUERY = "t:creature sort:edhrec"
LIMIT = 500

In [2]:
import mtgstats.scryfall.api as api
import mtgstats.cards as c
from mtgstats.analysis import cmc_analysis, color_analysis, subtype_analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
query = api.search(QUERY, limit=LIMIT)
cards = c.parse_cards(query)

In [4]:
# Go for the Throat - Non-artifact
# Doomblade - non-black
# Malicious Afflitcion - non-black
# Victim of Night - non-Vampire, non-Werewolf, non-Zombie
# Vendetta - non-black, can't regenerate, loses life equal to toughness
# Terror - Non-black
# Ultimate Price - Monocolored
# Devour in shadow - Can't regenerate, loses life equal to toughness
# Power Word Kill - non-Angel, non-Demon, non-Devil, non-Dragon

In [5]:
destructible = list(filter(lambda x: "Indestructible" not in x.get('keywords', []), cards))
c = color_analysis(destructible)
cmc, cum_cmc = cmc_analysis(destructible)
types = subtype_analysis(destructible)
data = pd.DataFrame(destructible)

Nissa, Vastwood Seer
Glasspool Mimic
Docent of Perfection
Birgi, God of Storytelling
Liliana, Heretical Healer
Jace, Vryn's Prodigy


In [6]:
# Número de criaturas não pretas: Doomblade, Malicious Affliction, Vendetta
non_black_creatures = len(data) - c['B']
print(non_black_creatures)

380


In [7]:
artifact_creatures =  len(data[data['type_line'].str.contains("Artifact")])
# Número de criaturas artefatos
print(len(data) - artifact_creatures)
# Terror
print(non_black_creatures - artifact_creatures)

441
341


In [8]:
# Numero de criaturas monocoloridas: Ultimate price
len(data[data.colors.str.len() == 1])

353

In [9]:
# Victim of Night
t = ['Zombie', 'Werewolf', 'Vampire']
len(data) - sum([types.get(x, 0) for x in t])

442

In [10]:
# Power Word Kill
t = ['Angel', 'Demon', 'Devil', 'Dragon']
len(data) - sum([types.get(x, 0) for x in t])

443

In [11]:
def tough_helper(x):
    t = x.get('toughness', 0)
    # Cartas com * na resistência vão ficar marcadas com -1
    return int(t) if t != '*' else 0

In [12]:
# Distribuição das resistências para Vendetta (criaturas não pretas)
t = list(
    map(tough_helper, 
        filter(lambda x: 'B' not in x.get('colors', []),
               destructible)))
from collections import Counter
counter = Counter(t)
s = sorted(counter.items(), key=lambda x:x[0])

_, t = zip(*s)
# Soma cumulativa <resitencia>: <soma>
dict(zip(_,np.cumsum(t)))

{0: 26,
 1: 114,
 2: 205,
 3: 252,
 4: 302,
 5: 331,
 6: 360,
 7: 368,
 8: 369,
 9: 374,
 10: 375,
 11: 376,
 12: 379,
 13: 380}

In [13]:
# Distribuição das resistências para Malicious Afliction
t = list(
    map(tough_helper,destructible))
from collections import Counter
counter = Counter(t)
s = sorted(counter.items(), key=lambda x:x[0])

_, t = zip(*s)
# Soma cumulativa <resitencia>: <soma>
dict(zip(_,np.cumsum(t)))

{0: 27,
 1: 132,
 2: 249,
 3: 315,
 4: 386,
 5: 421,
 6: 458,
 7: 466,
 8: 469,
 9: 474,
 10: 475,
 11: 476,
 12: 479,
 13: 480}